In [1]:
# 모델 구축
# 입력값 토큰화, 임베딩
# 훈련을 정의하고 학습
# 평가
# 예측하기 위한 데이터를 토큰화, 임베딩후 predict
# 라벨 출력

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

enc_IN = Input(shape=(4, 171)) # 영문 단어가 입력, 영문 단어는 4자짜리, 모든 문자의 수는 171개

# hidden state와 cell state를 전달해야 하므로 return_state=True 추가
_, state_h, state_c = LSTM(128, return_state=True)(enc_IN) 

# link = [state_h, state_c]
dec_IN = Input(shape=(3, 171)) # 한글 단어는 2자짜리+<Start>, 모든 문자의 수는 171개

# LSTM 모든 단의 출력을 사용해야 하므로 return_sequences=True 추가
dec_Y = LSTM(128, return_sequences=True)(dec_IN, initial_state=[state_h, state_c]) 

dec_OUT = Dense(171, activation='softmax')(dec_Y)
model = Model(inputs=[enc_IN, dec_IN], outputs=[dec_OUT])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 4, 171)]             0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 3, 171)]             0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 128),                153600    ['input_1[0][0]']             
                              (None, 128),                                                        
                              (None, 128)]                                                        
                                                                                              

In [10]:
import pandas as pd
import numpy as np

arr1 = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']
arr2 = pd.read_csv('korean.csv', header=None)
arr2 = arr2[0].values.tolist()
num_to_char = arr1 + arr2

In [11]:
char_to_num = {n: i for i, n in enumerate(num_to_char)}
print(len(char_to_num))

171


In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy') # loss만 출력

In [13]:
raw = pd.read_csv("translate.csv", header=None)
eng_kor = raw.values.tolist()
print(len(eng_kor))

110


In [21]:
def encode(eng_kor):
  enc_in = []
  dec_in = []
  rnn_out = []
  for seq in eng_kor:
    eng = [char_to_num[c] for c in seq[0]]
    enc_in.append(np.eye(171)[eng])

    kor = [char_to_num[c] for c in ('S'+seq[1])]
    dec_in.append(np.eye(171)[kor])

    target = [char_to_num[c] for c in (seq[1] + 'E')]
    rnn_out.append(target)

  enc_in = np.array(enc_in)
  dec_in = np.array(dec_in)
  rnn_out = np.array(rnn_out)

  rnn_out = np.expand_dims(rnn_out, axis=2)

  return enc_in, dec_in, rnn_out

In [23]:
sample = [['word', '단어']]
enc_in, dec_in, rnn_out = encode(sample)
# print(enc_in)

In [24]:
X_enc, X_dec, Y_rnn = encode(eng_kor)

In [25]:
%%time
model.fit([X_enc, X_dec], Y_rnn, epochs=500)

Epoch 1/500
4/4 [==============================] - 2s 36ms/step - loss: 5.1377
Epoch 2/500
4/4 [==============================] - 0s 6ms/step - loss: 5.1025
Epoch 3/500
4/4 [==============================] - 0s 5ms/step - loss: 5.0584
Epoch 4/500
4/4 [==============================] - 0s 6ms/step - loss: 4.9862
Epoch 5/500
4/4 [==============================] - 0s 6ms/step - loss: 4.8477
Epoch 6/500
4/4 [==============================] - 0s 6ms/step - loss: 4.5509
Epoch 7/500
4/4 [==============================] - 0s 6ms/step - loss: 3.9425
Epoch 8/500
4/4 [==============================] - 0s 6ms/step - loss: 3.4925
Epoch 9/500
4/4 [==============================] - 0s 6ms/step - loss: 3.6131
Epoch 10/500
4/4 [==============================] - 0s 6ms/step - loss: 3.4660
Epoch 11/500
4/4 [==============================] - 0s 6ms/step - loss: 3.3768
Epoch 12/500
4/4 [==============================] - 0s 6ms/step - loss: 3.3855
Epoch 13/500
4/4 [==============================] - 0s 6ms/s

In [27]:
from numpy.random import randint

pick = randint(0, len(eng_kor), 5)
choose = [eng_kor[i] for i in pick]

In [28]:
enc_in, dec_in, _ = encode(choose)
pred = model.predict([enc_in, dec_in])
print("예측 결과의 모양:", pred.shape)

1/1 [==============================] - 0s 383ms/step
예측 결과의 모양: (5, 3, 171)


In [29]:
for i in range(len(choose)):
  eng = choose[i][0]
  word = np.argmax(pred[i], axis=1)
  kor = ''
  for j in range(2):
    kor = kor + num_to_char[word[j]]
  print(eng, '->', kor)

meal -> 식사
them -> 그들
give -> 주다
meat -> 고기
fish -> 생선
